In [12]:
import pandas as pd
import pandas_datareader as pdr
import FinanceDataReader as fdr
 
stock_type = {
    'kospi': 'stockMkt',
    'kosdaq': 'kosdaqMkt'
}
 
class StockCodes: 
    def __init__(self, mt=None):
        self.get_method = mt
        if self.get_method is None:
            self.stock_codes()
        else:
            self.codes2 = fdr.StockListing('KRX')[['Market','Name','Symbol']]
            self.codes2 = self.codes2.rename(columns={'Market':'market','Name': 'name', 'Symbol': 'code'})

    def krx_stock_codes(self, market_type='kospi'):
        download_link = 'http://kind.krx.co.kr/corpgeneral/corpList.do'
        download_link = download_link + '?method=download'
        download_link = download_link + '&marketType=' + stock_type[market_type]
        df = pd.read_html(download_link, header=0)[0]
        df['market'] = 'KS' if market_type == 'kospi' else 'KQ'
        return df

    def get_code(self, name, target=None):
        code = self.codes.query(f"name=='{name}'")
        if len(code.index) == 0:
            return '000000'
        if target:
            code = f'{code.code.item():06d}.{code.market.item()}'
        else:
            if self.get_method is None:
                code = f'{code.code.item():06d}'
            else:
                code = f'{code.code.item()}'
        return code

    def find(self, name):
        str_expr = f"name.str.contains('{name}')"
        print(str_expr)
        code = self.codes.query(str_expr, engine='python')
        return code

    def stock_codes(self):
        kospi = self.krx_stock_codes('kospi')
        kosdaq = self.krx_stock_codes('kosdaq')

        self.codes = pd.concat([kospi, kosdaq], ignore_index=True)
        self.codes = self.codes[['market','회사명', '종목코드']]
        self.codes = self.codes.rename(columns={'회사명': 'name', '종목코드': 'code'})

    #    url = f'http://media.kisline.com/highlight/mainHighlight.nice?nav=1&paper_stock={code.code.item():06d}'
    def getFinancefromNaver(self, name):
        code = self.codes.query(f"name=='{name}'")
        url = f'http://finance.naver.com/item/main.nhn?code={code.code.item():06d}'
        print(url)
        tables = pd.read_html(url, encoding='euc-kr')[3]
        tables.set_index(('주요재무정보', '주요재무정보', '주요재무정보'), inplace=True)
        tables.index.rename('주요재무정보', inplace=True)
        tables.columns = tables.columns.droplevel(2)
        tables.columns = tables.columns.droplevel(0)
        return tables
 
 
def readData(name, start=None, end=None):
    data = fdr.DataReader(stock_codes.get_code(name), start, end)
    #data = pdr.get_data_yahoo(get_code(name))

    data.loc[(data[['High','Low','Open']] == 0).any(axis=1), ('High','Low','Open')] = data['Close']
    
    timep = 240
    data['high'] = data['High'].rolling(window=timep).max() 
    data['low'] = data['Low'].rolling(window=timep).min() 
    data['enval'] = data['Close'].rolling(window=20).mean() 
    data['mav120'] = data['Close'].rolling(window=120).mean() 
    data['mav240'] = data['Close'].rolling(window=240).mean() 
    data['e_high'] = data['enval'] * 1.2 
    data['e_low'] = data['enval'] * 0.8
    data['madi50'] = data['low'] + (data['high'] - data['low']) * 0.5
    data['madi60'] = data['low'] + (data['high'] - data['low']) * 0.4
    data['madi70'] = data['low'] + (data['high'] - data['low']) * 0.3
    data['madi80'] = data['low'] + (data['high'] - data['low']) * 0.2
    data['madi90'] = data['low'] + (data['high'] - data['low']) * 0.1
    data['v_high'] = data['Volume'].rolling(window=timep).max() 
    data['rate'] = (data['Close'] - data['low']) / (data['high'] - data['low'])
    data['diff'] = data['high'] / data['low']

    print("High:", data['high'][-1], "Low:", data['low'][-1], "diff:", (data['high'][-1]/ data['low'][-1]) * 100)
    print("current:", data['rate'][-1])
    print("Close:", data['Close'][-1], "medi_50:", data['madi50'][-1])
    return data
 
 
%matplotlib inline 
import pandas as pd
import mplfinance as mpf
 
def drawchart(name, range=200):

    tdata = readData(name, '2017')
    data = tdata[-range:]

    #fig = mpf.figure(style='yahoo',tight_layout=True, figsize=(12,5))
    #ax1 = fig.add_subplot(2,1,1)
    #ax2 = fig.add_subplot(2,1,2)  

    ap0 = [
        mpf.make_addplot(data[['madi60', 'madi70', 'madi80', 'madi90']], color='blue', width=0.5),
        mpf.make_addplot(data[['madi50']],color='g', width=1.5),
        mpf.make_addplot(data[['mav120']],color='teal', width=1),
        mpf.make_addplot(data[['mav240']],color='orange', width=1),
        mpf.make_addplot(data[['high', 'low']],color='c', width=0.5),
        mpf.make_addplot(data[['enval']],color='red', width=1),
        mpf.make_addplot(data[['e_high', 'e_low']],color='black', width=0.5),
        mpf.make_addplot(data[['v_high']],color='black', panel=1, ylabel='Volume', width=0.5),
        mpf.make_addplot(data[['rate']],color='red', panel=2, ylabel='rate', width=0.5),
        mpf.make_addplot(data[['diff']],color='green', panel=2, width=0.5),
        ]

    mc = mpf.make_marketcolors(
                              up='red',down='blue',
                              inherit=True
                            )
    s  = mpf.make_mpf_style(base_mpf_style="yahoo", edgecolor='black', marketcolors=mc)
    kwargs = dict(type='candle',
            #title=stock_codes.get_code(name),
            ylabel='Price',
            figratio=(4,2),
            panel_ratios=(10,2),
            addplot=ap0)

    mpf.plot(data, **kwargs, volume=True, style = s)
 
stock_codes = StockCodes()

In [1]:
s_name = 'SK가스'
drawchart(s_name,500)
stock_codes.getFinancefromNaver(s_name)

NameError: name 'drawchart' is not defined

In [4]:
stock_codes.codes

,market,name,code
0,KS,DRB동일,4840
1,KS,DSR,155660
2,KS,GS,78930
3,KS,GS글로벌,1250
4,KS,HDC현대산업개발,294870
...,...,...,...
2338,KQ,한컴위드,54920
2339,KQ,현대사료,16790
2340,KQ,홈센타홀딩스,60560
2341,KQ,휴맥스홀딩스,28080
